In [6]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

EPOCHS = 20
IMG_WIDTH = 224
IMG_HEIGHT = 224
NUM_CATEGORIES = 2
TEST_SIZE = 0.2

In [7]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels=[]
    counter = 0
    for labelname in os.listdir(data_dir):
        path= data_dir + "/" + labelname 
        for pic in os.listdir(path):
            path2= data_dir + "/" + labelname + "/" + pic
            img= cv2.imread(path2)
            #print ("path- ", path)
            #print ("path2- ", path2)
            re_img=cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
            images.append(re_img)
            labels.append(labelname)
            counter= counter+1
    print(counter) 
    return (images,labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
        """
# Create a convolutional neural network
    model = tf.keras.models.Sequential([

         # Convolutional layer. Learn 32 filters using a 3x3 kernel
         tf.keras.layers.Conv2D(
             32, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(4, 4)),
         tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(
             64, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
         tf.keras.layers.Dropout(0.2),
         

         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
        
        

         # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*32, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        
         
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        

        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*8, activation="relu"),
         tf.keras.layers.Dropout(0.2),
         
         # Add an output layer with number of categories
         tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
     ])

# Train neural network
    model.compile(
         optimizer="adam",
         loss="categorical_crossentropy",
         metrics=["accuracy"]
   )

    return model



In [8]:
# Get image arrays and labels for all image files
    images, labels = load_data("/home/ec2-user/SageMaker/BrainScans/brain")

    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE)


253


In [45]:
'''# Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*32, activation="relu"),
         tf.keras.layers.Dropout(0.2),
         
         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        '''

'# Add a hidden layer with dropout\n         tf.keras.layers.Dense(NUM_CATEGORIES*32, activation="relu"),\n         tf.keras.layers.Dropout(0.2),\n         \n         # Flatten units\n         tf.keras.layers.Flatten(),\n         tf.keras.layers.Dropout(0.2),\n        \n        # Add a hidden layer with dropout\n         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),\n         tf.keras.layers.Dropout(0.2),\n        \n        # Add a hidden layer with dropout\n         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),\n         tf.keras.layers.Dropout(0.2),\n        '

In [9]:
model = get_model()


In [10]:
import time

start = time.time()

# Fit model on training data
model_history = model.fit(x_train, y_train, epochs=EPOCHS)

end = time.time()
print(end - start)

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[32,32,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/conv2d_2/Relu (defined at <ipython-input-10-4c83784ca44c>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_2567]

Function call stack:
train_function


In [6]:
   # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

51/51 - 0s - loss: 0.4696 - acc: 0.7843


[0.46960273209740133, 0.78431374]

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

In [8]:
model.save("model-new")

In [14]:
!wget https://github.com/orislasky/CNN/blob/master/0/1%20no.jpeg

--2020-12-12 16:19:16--  https://github.com/orislasky/CNN/blob/master/0/1%20no.jpeg
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘1 no.jpeg.6’

1 no.jpeg.6             [ <=>                ]  88.38K  --.-KB/s    in 0.02s   

2020-12-12 16:19:16 (4.09 MB/s) - ‘1 no.jpeg.6’ saved [90506]



In [10]:
!pwd
from PIL import Image

/home/ec2-user/SageMaker/BrainScans


In [13]:
!ls /home/ec2-user/SageMaker/BrainScans/


10_no_rgb.jpg
165566_377107_bundle_archive.zip
17abe9584e3bb9b813aa8b7e857815a3_31692eea-70e6-4654-b5cf-377653fafd38.png
1 no.jpeg
1 no.jpeg.1
1 no.jpeg.2
1 no.jpeg.3
1 no.jpeg.4
1 no.jpeg.5
brain
BrainScans.ipynb
brain.tar.gz
brain_tumor_dataset
BrainTumor.jpg
color.jpeg
download
goldfinch_detections.png
gtsrb
IM-0006-0022.jpg
image-size1.png
image-size.jpg
image-size.png
Inference_BW.ipynb
Inference_Color_Works.ipynb
Inference.ipynb
model
model-new
new_image1.png
new_image.png
new-sized.jpeg
normal-brain-mri-4.jpg
object_detection_birds.ipynb
playback-fm_colorize-photo_a9a6aee290afc3b84ebca689cd37e0ed.jpg
playback-fm_colorize-photo_b44eb41b725ca6a61162e3e5ca696aca.jpeg
README.md
reset_gpu_memory.ipynb
Screen Shot 2020-11-07 at 19.49.32.jpg
Screen Shot 2020-11-07 at 19.49.32.png
Screen Shot 2020-11-08 at 9.43.19 PM.jpg
Screen Shot 2020-11-08 at 9.43.19 PM.png
Screen Shot 2020-11-08 at 9.55.04 PM.jpg
tools
TumorPic1.jpg
Untitled.ipynb
untitled.txt


In [39]:
image  = Image.open('/home/ec2-user/SageMaker/BrainScans/color.jpeg')

FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/object_detection_birds_2020-08-31/color.jpeg'

In [15]:
image  = Image.open('/home/ec2-user/SageMaker/BrainScans/1%20no.jpeg.jpeg')

FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/BrainScans/1%20no.jpeg.jpeg'

In [97]:
model = tf.keras.models.load_model('model/')

In [98]:
image.show()



In [99]:
image.save('new_image.png')
new_image = image.resize((224, 224))
new_image.save('image-size.png')

In [100]:
print(model.summary())
print("=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_60 (Dropout)         (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_61 (Dropout)         (None, 18, 18, 64)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 20736)            

In [101]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img

x = img_to_array(new_image)
x = np.expand_dims(x, axis=0)
preds = model.predict(x)
#prob = model.predict_proba(x)
print(preds)

[[0.47692132 0.5230786 ]]
